In [5]:
###Providing numerical evidence of the fidelity of a QDrift channel to reproduce spectrum in Alanine in either the longest time scale of the protocol (mixing time)
# or both the mixing time and t1/t2 timescales...

#TODO: fix the QDrift channel formulation!

import openfermion as of

import sys
sys.path.append('./utils/')

from basis_utils import Sz,Sx,Sz, Sy
from basis_utils import read_spinach_info, build_list_ISTs, NormalizeBasis, build_symbolic_list_ISTs, MatRepLib
from simulation_utils import GenH0_Ham, HamMatRep, GenNOESYSpectrum, sqcosbell_2d_apod, GenFIDsignals
import scipy.io as spio
#from 
from scipy.linalg import expm
import cirq
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import savemat
import pickle

from matplotlib.colors import TwoSlopeNorm



In [6]:
from scipy.linalg import sqrtm

#calculation o0f square roots in the Pauli basis 

#####Function to compute fidelities between density matrices...
#def Fid_DenMat(rho,sigma):
#    sqrt_


# QDrift implementation.

def Samp_Gates(ListGates,List_weights,N):
    #Return a QDrift product formula that simulates a target time T using N samples drawn from the probability distribution...

    Gamma = np.sum(List_weights)
    prob_dist = (1.0/Gamma)*np.array(List_weights)

    #Draw a sample of size N for the implementation of the product formula...
    values = np.arange(len(ListGates))
    samples = np.random.choice(values, size=N, p=prob_dist)
    
    #print("Indixes of gates sampled:",samples)
    drawn_jumps = []
    for i in range(len(samples)):
        if samples[i]!=0:
            drawn_jumps.append(samples[i])

    print("Indices of jump operators drwan during construction of QDrift channel", drawn_jumps)

    ProdF = np.copy(ListGates[samples[0]])
    for i in range(1,N):
        ProdF=ListGates[samples[i]]@ProdF

    return ProdF


def Normalize_and_weightOps(ListOps):
    """
    ListOps contains the generators of evolution, either the representation of the coherent part of evolution or the jump operators 
    """
    List_weights=[]

    for i in range(len(ListOps)):
        List_weights.append(np.max(np.abs(np.linalg.eigvals(ListOps[i]))))

    #Normalize the operator according to the weights...
    Norm_ops =[]

    for i in range(len(ListOps)):
        Norm_ops.append(ListOps[i]/List_weights[i])

    return Norm_ops, List_weights


def QDrift(H0,List_jumps,T,N):


    Norm_ops, List_weights= Normalize_and_weightOps([H0]+List_jumps)

    ###Definition of gates...
    List_Gates = []

    #for i in range(len(List_jumps)):
    #    List_weights.append(rates[i])
    Gamma = np.sum(np.array(List_weights))

    deltaT = T*Gamma/N

    #List_Gates.append(expm(-1j*H0*deltaT))

    for i in range(len(Norm_ops)):
        if i==0:###I assume that the first element of the operators correspond to H0!!!
            List_Gates.append(expm(-1j*Norm_ops[i]*deltaT))
        else:
            List_Gates.append(expm(Norm_ops[i]*deltaT))
        

    ProdF = Samp_Gates(List_Gates,List_weights,N)

    return ProdF


def QDriftChann(List_gens,time_evol):
    """
    Args:
    List_gens: list of the generators of the time evolution, it is assumed that the first element is H_{0}
    list_probs: list of the corresponding probabilities to sample the exponentials of generators
    Gamma: sum of the weights of each of the normalized operators
    time_evol: the time for which we perform the propagation
    """

    Norm_ops, List_weights = Normalize_and_weightOps(List_gens)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    Qdrift_chann = pks[0]*expm(-1j*Norm_ops[0]*time_evol*Gamma)
    for i in range(1,len(pks)):
        Qdrift_chann = pks[i]*expm(Norm_ops[i]*time_evol*Gamma)

    return Qdrift_chann


def ProbSingleJumpLayers(List_gens,SingProbs,time_evol,Nsteps):

    deltT = time_evol/Nsteps
    
    JumpEv_ops = []
    for i in range(1,len(List_gens)):
        JumpEv_ops.append(expm(List_gens[i]*deltT))


    Glob_diss_chann = np.zeros_like(JumpEv_ops[0])

    for n in range(Nsteps):
        R_Op = expm(-1j*n*deltT*List_gens[0])
        L_Op = expm(-1j*(Nsteps-n-1)*deltT*List_gens[0])

        for k in range(len(SingProbs)):
            Glob_diss_chann+=SingProbs[k]*L_Op@JumpEv_ops[k]@R_Op

    return Glob_diss_chann



def eff_QDriftChann_OneJump(List_gens,time_evol,NSteps):
    """
    Calculation of an effective QDrift channel. Based on the observation that the action of jump operators are in essence rare events 
    """
    Norm_ops, List_weights = Normalize_and_weightOps(List_gens)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    Eff_probs = np.zeros(len(List_gens))

    Eff_probs[0] = pks[0]**(NSteps) #probability of sampling coherent evolution only

    for i in range(1,len(Eff_probs)):
        Eff_probs[i] = pks[0]**(NSteps-1) * pks[i] #probability of sampling a circuit with NSteps-1 layers of H0 and one jump operator
    
    #Glob_Chann = Eff_probs[0]*expm(-1j*List_gens[0]*time_evol)
    Glob_Chann = Eff_probs[0]*expm(-1j*Norm_ops[0]*time_evol*Gamma)

    #Glob_Chann = Glob_Chann+ProbSingleJumpLayers(List_gens,Eff_probs[1:],time_evol,NSteps)

    Glob_Chann = Glob_Chann+ProbSingleJumpLayers(Norm_ops,Eff_probs[1:],time_evol*Gamma,NSteps)
    return Glob_Chann


def eff_QDriftChann_InitJump(List_gens,time_evol,NSteps):
    """
    Calculation of an effective QDrift channel. Based on the observation that the action of jump operators are in essence rare events 
    """
    Norm_ops, List_weights = Normalize_and_weightOps(List_gens)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    Eff_probs = np.zeros(len(List_gens))

    Eff_probs[0] = pks[0]**(NSteps) #probability of sampling coherent evolution only

    for i in range(1,len(Eff_probs)):
        Eff_probs[i] = pks[0]**(NSteps-1) * pks[i] #probability of sampling a circuit with NSteps-1 layers of H0 and one jump operator
    
    #Glob_Chann = Eff_probs[0]*expm(-1j*List_gens[0]*time_evol)
    Glob_Chann = Eff_probs[0]*expm(-1j*Norm_ops[0]*time_evol*Gamma)

    for i in range(1,len(Eff_probs)):
        Glob_Chann = Glob_Chann + NSteps*Eff_probs[i]*expm(-1j*Norm_ops[0]*(time_evol/NSteps)*Gamma*(NSteps-1))*expm(Norm_ops[i]*(time_evol/NSteps)*Gamma)

    #Glob_Chann =Glob_Chann + Eff_probs[0]

    #Glob_Chann = Glob_Chann+ProbSingleJumpLayers(List_gens,Eff_probs[1:],time_evol,NSteps)

    #Glob_Chann = Glob_Chann+ProbSingleJumpLayers(Norm_ops,Eff_probs[1:],time_evol*Gamma,NSteps)
    return Glob_Chann


###Generation of QDrift-based NOESY spectrum..

def GenFID_TmixQDriftsignals(Ham,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    #This is the convention for our Trotter step: apply first the coherent part of the super-Liouvillian operator, followed by the jump operators in the order they appear in the array...
    R = sum(List_jumps)

    L_dt1 = expm((-1j*Ham+R)*dt1)

    L_dt2 = expm((-1j*Ham+R)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90y@pulse_mix@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_mix@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_mix@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_mix@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4


def GenFID_TmixQDrift_cohT1T2(Ham,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    
    R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90y@pulse_mix@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_mix@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_mix@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_mix@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4


def GenFID_TmixInitJumpQDrift_cohT1T2(Ham,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    
    R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90y@pulse_mix@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_mix@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_mix@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_mix@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4





def GenFID_DissTmix(Ham,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    
    R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham+R)*dt1)

    L_dt2 = expm((-1j*Ham+R)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    pulse_mix = expm(R*tmix) #eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    print("Difference between pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90y@pulse_mix@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_mix@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_mix@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_mix@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4



    



In [7]:
text4="""1      (0,0)   (0,0)   (0,0)   (0,0)   
  2      (0,0)   (0,0)   (0,0)   (1,1)   
  3      (0,0)   (0,0)   (0,0)   (1,0)   
  4      (0,0)   (0,0)   (0,0)   (1,-1)  
  5      (0,0)   (0,0)   (1,1)   (0,0)   
  6      (0,0)   (0,0)   (1,1)   (1,1)   
  7      (0,0)   (0,0)   (1,1)   (1,0)   
  8      (0,0)   (0,0)   (1,1)   (1,-1)  
  9      (0,0)   (0,0)   (1,0)   (0,0)   
  10     (0,0)   (0,0)   (1,0)   (1,1)   
  11     (0,0)   (0,0)   (1,0)   (1,0)   
  12     (0,0)   (0,0)   (1,0)   (1,-1)  
  13     (0,0)   (0,0)   (1,-1)  (0,0)   
  14     (0,0)   (0,0)   (1,-1)  (1,1)   
  15     (0,0)   (0,0)   (1,-1)  (1,0)   
  16     (0,0)   (0,0)   (1,-1)  (1,-1)  
  17     (0,0)   (1,1)   (0,0)   (0,0)   
  18     (0,0)   (1,1)   (0,0)   (1,1)   
  19     (0,0)   (1,1)   (0,0)   (1,0)   
  20     (0,0)   (1,1)   (0,0)   (1,-1)  
  21     (0,0)   (1,1)   (1,1)   (0,0)   
  22     (0,0)   (1,1)   (1,1)   (1,1)   
  23     (0,0)   (1,1)   (1,1)   (1,0)   
  24     (0,0)   (1,1)   (1,1)   (1,-1)  
  25     (0,0)   (1,1)   (1,0)   (0,0)   
  26     (0,0)   (1,1)   (1,0)   (1,1)   
  27     (0,0)   (1,1)   (1,0)   (1,0)   
  28     (0,0)   (1,1)   (1,0)   (1,-1)  
  29     (0,0)   (1,1)   (1,-1)  (0,0)   
  30     (0,0)   (1,1)   (1,-1)  (1,1)   
  31     (0,0)   (1,1)   (1,-1)  (1,0)   
  32     (0,0)   (1,1)   (1,-1)  (1,-1)  
  33     (0,0)   (1,0)   (0,0)   (0,0)   
  34     (0,0)   (1,0)   (0,0)   (1,1)   
  35     (0,0)   (1,0)   (0,0)   (1,0)   
  36     (0,0)   (1,0)   (0,0)   (1,-1)  
  37     (0,0)   (1,0)   (1,1)   (0,0)   
  38     (0,0)   (1,0)   (1,1)   (1,1)   
  39     (0,0)   (1,0)   (1,1)   (1,0)   
  40     (0,0)   (1,0)   (1,1)   (1,-1)  
  41     (0,0)   (1,0)   (1,0)   (0,0)   
  42     (0,0)   (1,0)   (1,0)   (1,1)   
  43     (0,0)   (1,0)   (1,0)   (1,0)   
  44     (0,0)   (1,0)   (1,0)   (1,-1)  
  45     (0,0)   (1,0)   (1,-1)  (0,0)   
  46     (0,0)   (1,0)   (1,-1)  (1,1)   
  47     (0,0)   (1,0)   (1,-1)  (1,0)   
  48     (0,0)   (1,0)   (1,-1)  (1,-1)  
  49     (0,0)   (1,-1)  (0,0)   (0,0)   
  50     (0,0)   (1,-1)  (0,0)   (1,1)   
  51     (0,0)   (1,-1)  (0,0)   (1,0)   
  52     (0,0)   (1,-1)  (0,0)   (1,-1)  
  53     (0,0)   (1,-1)  (1,1)   (0,0)   
  54     (0,0)   (1,-1)  (1,1)   (1,1)   
  55     (0,0)   (1,-1)  (1,1)   (1,0)   
  56     (0,0)   (1,-1)  (1,1)   (1,-1)  
  57     (0,0)   (1,-1)  (1,0)   (0,0)   
  58     (0,0)   (1,-1)  (1,0)   (1,1)   
  59     (0,0)   (1,-1)  (1,0)   (1,0)   
  60     (0,0)   (1,-1)  (1,0)   (1,-1)  
  61     (0,0)   (1,-1)  (1,-1)  (0,0)   
  62     (0,0)   (1,-1)  (1,-1)  (1,1)   
  63     (0,0)   (1,-1)  (1,-1)  (1,0)   
  64     (0,0)   (1,-1)  (1,-1)  (1,-1)  
  65     (1,1)   (0,0)   (0,0)   (0,0)   
  66     (1,1)   (0,0)   (0,0)   (1,1)   
  67     (1,1)   (0,0)   (0,0)   (1,0)   
  68     (1,1)   (0,0)   (0,0)   (1,-1)  
  69     (1,1)   (0,0)   (1,1)   (0,0)   
  70     (1,1)   (0,0)   (1,1)   (1,1)   
  71     (1,1)   (0,0)   (1,1)   (1,0)   
  72     (1,1)   (0,0)   (1,1)   (1,-1)  
  73     (1,1)   (0,0)   (1,0)   (0,0)   
  74     (1,1)   (0,0)   (1,0)   (1,1)   
  75     (1,1)   (0,0)   (1,0)   (1,0)   
  76     (1,1)   (0,0)   (1,0)   (1,-1)  
  77     (1,1)   (0,0)   (1,-1)  (0,0)   
  78     (1,1)   (0,0)   (1,-1)  (1,1)   
  79     (1,1)   (0,0)   (1,-1)  (1,0)   
  80     (1,1)   (0,0)   (1,-1)  (1,-1)  
  81     (1,1)   (1,1)   (0,0)   (0,0)   
  82     (1,1)   (1,1)   (0,0)   (1,1)   
  83     (1,1)   (1,1)   (0,0)   (1,0)   
  84     (1,1)   (1,1)   (0,0)   (1,-1)  
  85     (1,1)   (1,1)   (1,1)   (0,0)   
  86     (1,1)   (1,1)   (1,1)   (1,1)   
  87     (1,1)   (1,1)   (1,1)   (1,0)   
  88     (1,1)   (1,1)   (1,1)   (1,-1)  
  89     (1,1)   (1,1)   (1,0)   (0,0)   
  90     (1,1)   (1,1)   (1,0)   (1,1)   
  91     (1,1)   (1,1)   (1,0)   (1,0)   
  92     (1,1)   (1,1)   (1,0)   (1,-1)  
  93     (1,1)   (1,1)   (1,-1)  (0,0)   
  94     (1,1)   (1,1)   (1,-1)  (1,1)   
  95     (1,1)   (1,1)   (1,-1)  (1,0)   
  96     (1,1)   (1,1)   (1,-1)  (1,-1)  
  97     (1,1)   (1,0)   (0,0)   (0,0)   
  98     (1,1)   (1,0)   (0,0)   (1,1)   
  99     (1,1)   (1,0)   (0,0)   (1,0)   
  100    (1,1)   (1,0)   (0,0)   (1,-1)  
  101    (1,1)   (1,0)   (1,1)   (0,0)   
  102    (1,1)   (1,0)   (1,1)   (1,1)   
  103    (1,1)   (1,0)   (1,1)   (1,0)   
  104    (1,1)   (1,0)   (1,1)   (1,-1)  
  105    (1,1)   (1,0)   (1,0)   (0,0)   
  106    (1,1)   (1,0)   (1,0)   (1,1)   
  107    (1,1)   (1,0)   (1,0)   (1,0)   
  108    (1,1)   (1,0)   (1,0)   (1,-1)  
  109    (1,1)   (1,0)   (1,-1)  (0,0)   
  110    (1,1)   (1,0)   (1,-1)  (1,1)   
  111    (1,1)   (1,0)   (1,-1)  (1,0)   
  112    (1,1)   (1,0)   (1,-1)  (1,-1)  
  113    (1,1)   (1,-1)  (0,0)   (0,0)   
  114    (1,1)   (1,-1)  (0,0)   (1,1)   
  115    (1,1)   (1,-1)  (0,0)   (1,0)   
  116    (1,1)   (1,-1)  (0,0)   (1,-1)  
  117    (1,1)   (1,-1)  (1,1)   (0,0)   
  118    (1,1)   (1,-1)  (1,1)   (1,1)   
  119    (1,1)   (1,-1)  (1,1)   (1,0)   
  120    (1,1)   (1,-1)  (1,1)   (1,-1)  
  121    (1,1)   (1,-1)  (1,0)   (0,0)   
  122    (1,1)   (1,-1)  (1,0)   (1,1)   
  123    (1,1)   (1,-1)  (1,0)   (1,0)   
  124    (1,1)   (1,-1)  (1,0)   (1,-1)  
  125    (1,1)   (1,-1)  (1,-1)  (0,0)   
  126    (1,1)   (1,-1)  (1,-1)  (1,1)   
  127    (1,1)   (1,-1)  (1,-1)  (1,0)   
  128    (1,1)   (1,-1)  (1,-1)  (1,-1)  
  129    (1,0)   (0,0)   (0,0)   (0,0)   
  130    (1,0)   (0,0)   (0,0)   (1,1)   
  131    (1,0)   (0,0)   (0,0)   (1,0)   
  132    (1,0)   (0,0)   (0,0)   (1,-1)  
  133    (1,0)   (0,0)   (1,1)   (0,0)   
  134    (1,0)   (0,0)   (1,1)   (1,1)   
  135    (1,0)   (0,0)   (1,1)   (1,0)   
  136    (1,0)   (0,0)   (1,1)   (1,-1)  
  137    (1,0)   (0,0)   (1,0)   (0,0)   
  138    (1,0)   (0,0)   (1,0)   (1,1)   
  139    (1,0)   (0,0)   (1,0)   (1,0)   
  140    (1,0)   (0,0)   (1,0)   (1,-1)  
  141    (1,0)   (0,0)   (1,-1)  (0,0)   
  142    (1,0)   (0,0)   (1,-1)  (1,1)   
  143    (1,0)   (0,0)   (1,-1)  (1,0)   
  144    (1,0)   (0,0)   (1,-1)  (1,-1)  
  145    (1,0)   (1,1)   (0,0)   (0,0)   
  146    (1,0)   (1,1)   (0,0)   (1,1)   
  147    (1,0)   (1,1)   (0,0)   (1,0)   
  148    (1,0)   (1,1)   (0,0)   (1,-1)  
  149    (1,0)   (1,1)   (1,1)   (0,0)   
  150    (1,0)   (1,1)   (1,1)   (1,1)   
  151    (1,0)   (1,1)   (1,1)   (1,0)   
  152    (1,0)   (1,1)   (1,1)   (1,-1)  
  153    (1,0)   (1,1)   (1,0)   (0,0)   
  154    (1,0)   (1,1)   (1,0)   (1,1)   
  155    (1,0)   (1,1)   (1,0)   (1,0)   
  156    (1,0)   (1,1)   (1,0)   (1,-1)  
  157    (1,0)   (1,1)   (1,-1)  (0,0)   
  158    (1,0)   (1,1)   (1,-1)  (1,1)   
  159    (1,0)   (1,1)   (1,-1)  (1,0)   
  160    (1,0)   (1,1)   (1,-1)  (1,-1)  
  161    (1,0)   (1,0)   (0,0)   (0,0)   
  162    (1,0)   (1,0)   (0,0)   (1,1)   
  163    (1,0)   (1,0)   (0,0)   (1,0)   
  164    (1,0)   (1,0)   (0,0)   (1,-1)  
  165    (1,0)   (1,0)   (1,1)   (0,0)   
  166    (1,0)   (1,0)   (1,1)   (1,1)   
  167    (1,0)   (1,0)   (1,1)   (1,0)   
  168    (1,0)   (1,0)   (1,1)   (1,-1)  
  169    (1,0)   (1,0)   (1,0)   (0,0)   
  170    (1,0)   (1,0)   (1,0)   (1,1)   
  171    (1,0)   (1,0)   (1,0)   (1,0)   
  172    (1,0)   (1,0)   (1,0)   (1,-1)  
  173    (1,0)   (1,0)   (1,-1)  (0,0)   
  174    (1,0)   (1,0)   (1,-1)  (1,1)   
  175    (1,0)   (1,0)   (1,-1)  (1,0)   
  176    (1,0)   (1,0)   (1,-1)  (1,-1)  
  177    (1,0)   (1,-1)  (0,0)   (0,0)   
  178    (1,0)   (1,-1)  (0,0)   (1,1)   
  179    (1,0)   (1,-1)  (0,0)   (1,0)   
  180    (1,0)   (1,-1)  (0,0)   (1,-1)  
  181    (1,0)   (1,-1)  (1,1)   (0,0)   
  182    (1,0)   (1,-1)  (1,1)   (1,1)   
  183    (1,0)   (1,-1)  (1,1)   (1,0)   
  184    (1,0)   (1,-1)  (1,1)   (1,-1)  
  185    (1,0)   (1,-1)  (1,0)   (0,0)   
  186    (1,0)   (1,-1)  (1,0)   (1,1)   
  187    (1,0)   (1,-1)  (1,0)   (1,0)   
  188    (1,0)   (1,-1)  (1,0)   (1,-1)  
  189    (1,0)   (1,-1)  (1,-1)  (0,0)   
  190    (1,0)   (1,-1)  (1,-1)  (1,1)   
  191    (1,0)   (1,-1)  (1,-1)  (1,0)   
  192    (1,0)   (1,-1)  (1,-1)  (1,-1)  
  193    (1,-1)  (0,0)   (0,0)   (0,0)   
  194    (1,-1)  (0,0)   (0,0)   (1,1)   
  195    (1,-1)  (0,0)   (0,0)   (1,0)   
  196    (1,-1)  (0,0)   (0,0)   (1,-1)  
  197    (1,-1)  (0,0)   (1,1)   (0,0)   
  198    (1,-1)  (0,0)   (1,1)   (1,1)   
  199    (1,-1)  (0,0)   (1,1)   (1,0)   
  200    (1,-1)  (0,0)   (1,1)   (1,-1)  
  201    (1,-1)  (0,0)   (1,0)   (0,0)   
  202    (1,-1)  (0,0)   (1,0)   (1,1)   
  203    (1,-1)  (0,0)   (1,0)   (1,0)   
  204    (1,-1)  (0,0)   (1,0)   (1,-1)  
  205    (1,-1)  (0,0)   (1,-1)  (0,0)   
  206    (1,-1)  (0,0)   (1,-1)  (1,1)   
  207    (1,-1)  (0,0)   (1,-1)  (1,0)   
  208    (1,-1)  (0,0)   (1,-1)  (1,-1)  
  209    (1,-1)  (1,1)   (0,0)   (0,0)   
  210    (1,-1)  (1,1)   (0,0)   (1,1)   
  211    (1,-1)  (1,1)   (0,0)   (1,0)   
  212    (1,-1)  (1,1)   (0,0)   (1,-1)  
  213    (1,-1)  (1,1)   (1,1)   (0,0)   
  214    (1,-1)  (1,1)   (1,1)   (1,1)   
  215    (1,-1)  (1,1)   (1,1)   (1,0)   
  216    (1,-1)  (1,1)   (1,1)   (1,-1)  
  217    (1,-1)  (1,1)   (1,0)   (0,0)   
  218    (1,-1)  (1,1)   (1,0)   (1,1)   
  219    (1,-1)  (1,1)   (1,0)   (1,0)   
  220    (1,-1)  (1,1)   (1,0)   (1,-1)  
  221    (1,-1)  (1,1)   (1,-1)  (0,0)   
  222    (1,-1)  (1,1)   (1,-1)  (1,1)   
  223    (1,-1)  (1,1)   (1,-1)  (1,0)   
  224    (1,-1)  (1,1)   (1,-1)  (1,-1)  
  225    (1,-1)  (1,0)   (0,0)   (0,0)   
  226    (1,-1)  (1,0)   (0,0)   (1,1)   
  227    (1,-1)  (1,0)   (0,0)   (1,0)   
  228    (1,-1)  (1,0)   (0,0)   (1,-1)  
  229    (1,-1)  (1,0)   (1,1)   (0,0)   
  230    (1,-1)  (1,0)   (1,1)   (1,1)   
  231    (1,-1)  (1,0)   (1,1)   (1,0)   
  232    (1,-1)  (1,0)   (1,1)   (1,-1)  
  233    (1,-1)  (1,0)   (1,0)   (0,0)   
  234    (1,-1)  (1,0)   (1,0)   (1,1)   
  235    (1,-1)  (1,0)   (1,0)   (1,0)   
  236    (1,-1)  (1,0)   (1,0)   (1,-1)  
  237    (1,-1)  (1,0)   (1,-1)  (0,0)   
  238    (1,-1)  (1,0)   (1,-1)  (1,1)   
  239    (1,-1)  (1,0)   (1,-1)  (1,0)   
  240    (1,-1)  (1,0)   (1,-1)  (1,-1)  
  241    (1,-1)  (1,-1)  (0,0)   (0,0)   
  242    (1,-1)  (1,-1)  (0,0)   (1,1)   
  243    (1,-1)  (1,-1)  (0,0)   (1,0)   
  244    (1,-1)  (1,-1)  (0,0)   (1,-1)  
  245    (1,-1)  (1,-1)  (1,1)   (0,0)   
  246    (1,-1)  (1,-1)  (1,1)   (1,1)   
  247    (1,-1)  (1,-1)  (1,1)   (1,0)   
  248    (1,-1)  (1,-1)  (1,1)   (1,-1)  
  249    (1,-1)  (1,-1)  (1,0)   (0,0)   
  250    (1,-1)  (1,-1)  (1,0)   (1,1)   
  251    (1,-1)  (1,-1)  (1,0)   (1,0)   
  252    (1,-1)  (1,-1)  (1,0)   (1,-1)  
  253    (1,-1)  (1,-1)  (1,-1)  (0,0)   
  254    (1,-1)  (1,-1)  (1,-1)  (1,1)   
  255    (1,-1)  (1,-1)  (1,-1)  (1,0)   
  256    (1,-1)  (1,-1)  (1,-1)  (1,-1)"""



In [45]:
of.commutator(Sz(0)*Sz(1)+Sy(0)*Sy(1)+Sx(0)*Sx(1),Sz(0)*Sz(2)+Sy(0)*Sy(2)+Sx(0)*Sx(2))

0.125j [X0 Y1 Z2] +
-0.125j [X0 Z1 Y2] +
-0.125j [Y0 X1 Z2] +
0.125j [Y0 Z1 X2] +
0.125j [Z0 X1 Y2] +
-0.125j [Z0 Y1 X2]

In [8]:
data_ala = read_spinach_info(text4)

basis_ala = build_list_ISTs(data_ala)
prefacts,Symb_ALA_basis = build_symbolic_list_ISTs(data_ala)

#Normbasis = NormalizeBasis(basis,n_qubits=4,checkOrth=True) I have verified the orthonormalization of the basis
Normbasis_ala = NormalizeBasis(basis_ala,n_qubits=4,checkOrth=False)
Normbasis_ala = np.array(Normbasis_ala)

f = open('./data/ALA_JOps.pk','rb')
dat = pickle.load(f)

JumpOps = dat['JOps']

Rtrunc = sum(JumpOps)


In [9]:
###The reference FID signals and spectrum
loadMat = spio.loadmat('./data/NOESYdata_ALA_withGradients.mat',squeeze_me=True)

t_grid1 = loadMat['p']['time_grid1'].item()
t_grid2 = loadMat['p']['time_grid2'].item()

R_ala = loadMat['p']['R'].item()
H_ala = loadMat['p']['H'].item().toarray()

rho0 = np.array(loadMat['p']['rho0'].item().toarray())

coil = np.array(loadMat['p']['coil'].item())

Lx = loadMat['p']['Lx'].item().toarray() 
Ly = loadMat['p']['Ly'].item().toarray() 

###Dynamical evolution for calculation of 2D spectra...
#Tpts1 = len(t_grid1)
#Tpts2 = len(t_grid2)

#Parameters taken from Spinach script
tmix = 1.0
dt1 = 5e-4
dt2 = 5e-4
#dt1 = 0.00512
#dt2 = 0.00512
Tpts1 = 512
Tpts2 = 512

##Parameters for Fourier transform
#zerofill1 = 4096
#zerofill2 = 4096

#R_trunc is going to be the sum of the matrix representation of the three jump operators
#fid_1_ref, fid_2_ref, fid_3_ref, fid_4_ref =  GenFIDsignals(H_ala,Rtrunc,Tpts1,Tpts2,rho0,coil,tmix,dt1,dt2,Lx,Ly)


In [68]:
#effQdriftTmix = eff_QDriftChann_OneJump([H_ala]+JumpOps,tmix,Ntmix)

In [71]:
#np.linalg.norm(expm((-1j*H_ala+Rtrunc)*tmix)-effQdriftTmix)

3.3026644544248986

In [70]:
np.linalg.norm(expm((-1j*H_ala+Rtrunc)*tmix)-expm(-1j*H_ala*tmix))

3.8675275638659836

In [84]:
####Some understanding of the QDrift channel...

ListOps = [H_ala]+JumpOps

Norm_ops, List_weights = Normalize_and_weightOps(ListOps)


#probabilites for gate sampling
pks = np.array(np.copy(List_weights))
Gamma = np.sum(pks)
pks = (1.0/Gamma)*pks



In [85]:
###What is a reasonable discretization timestep for mixing time assuming a QDrift implementation?
#we can identify the relevant timescale by identifying the highest frequency of oscillation of transversal polarization under free evolution...

trans_pol = expm(-1j*Lx*np.pi/2)@rho0
trans_pol = trans_pol.flatten()

eigvalsL,eigvectL = np.linalg.eig(H_ala+1j*Rtrunc)

overlaps = np.zeros(len(eigvalsL),dtype=complex)

for i in range(len(overlaps)):
    overlaps[i] =  np.dot(np.conj(eigvectL[:,i]),trans_pol)


idx_max = np.abs(overlaps).argmax()

tmix_disc_tscale = 1/np.real(eigvalsL[idx_max])

Ntmix_steps = int(np.ceil(tmix/tmix_disc_tscale))


In [86]:
List_weights/sum(List_weights)

array([9.99955205e-01, 1.53581459e-05, 1.48248770e-05, 1.46115694e-05])

In [87]:
###probability of sampling at least 1 jump operator:

1 - pks[0]**(Ntmix_steps)


0.1111770433453948

In [88]:
#Probability of sampling jump operators 1,2 and 3...

Ntmix_steps*pks[0]**(Ntmix_steps-1)*pks[1],Ntmix_steps*pks[0]**(Ntmix_steps-1)*pks[2], Ntmix_steps*pks[0]**(Ntmix_steps-1)*pks[3]


(0.03591652871427717, 0.034669427022809174, 0.03417058634622201)

In [30]:
np.abs(overlaps).argmax()

192

In [31]:
np.abs(overlaps)[192]

0.21732502529005593

In [21]:
np.max(np.abs(np.linalg.eigvals(Norm_ops[3])))


1.0000000000000002

In [61]:
#Calculation of the FID signals through an effective QDrift channel:

Ntmix = 2631
T1 = dt1*Tpts1
T2 = dt2*Tpts2


fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4 = GenFID_TmixQDriftsignals(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)


Number of Trotter steps for deterministic simulation of T1 512
Number of Trotter steps for deterministic simulation of T2 512
Number of Trotter steps for QDrift simulation of tmix 2631
Difference between Trotterized pulse_mix and reference pulse_mix: 3.0162096461998376


/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1932871311.py:262: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1932871311.py:265: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1932871311.py:268: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_tem

In [75]:
####Saving FID for spectrum plotting....

cos_fid = fid_temp_1 - fid_temp_3
sin_fid = fid_temp_2 - fid_temp_4

ref_cos_fid = fid_1_ref - fid_3_ref
ref_sin_fid = fid_2_ref - fid_4_ref

savemat('ALA_truncR_FIDcos_QDrift512.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_QDrift512.mat', {'FID_sin': sin_fid})

savemat('ALA_truncR_FIDcos_Ref512.mat', {'FID_cos': ref_cos_fid})
savemat('ALA_truncR_FIDsin_Ref512.mat', {'FID_sin': ref_sin_fid})



# Coherent T1/T2 evolution

In [92]:
fid_coh_1, fid_coh_2, fid_coh_3, fid_coh_4 = GenFID_TmixQDrift_cohT1T2(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)

Number of Trotter steps for deterministic simulation of T1 512
Number of Trotter steps for deterministic simulation of T2 512
Number of Trotter steps for QDrift simulation of tmix 2631
Difference between Trotterized pulse_mix and reference pulse_mix: 3.3026644544248986


/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/2501787718.py:328: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/2501787718.py:331: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/2501787718.py:334: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_tem

In [93]:
cos_coh_fid = fid_coh_1 - fid_coh_3
sin_coh_fid = fid_coh_2 - fid_coh_4

savemat('ALA_truncR_FIDcos_QDrift512_cohT1T2.mat', {'FID_cos': cos_coh_fid})
savemat('ALA_truncR_FIDsin_QDrift512_cohT1T2.mat', {'FID_sin': sin_coh_fid})

#The spectrum only shows quantitative differences

# Coherent T1/T2 time evolution + 1 step QDrift channel

In [102]:
Ntmix = 1

fid_coh_1, fid_coh_2, fid_coh_3, fid_coh_4 = GenFID_TmixQDrift_cohT1T2(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)

Number of Trotter steps for deterministic simulation of T1 512
Number of Trotter steps for deterministic simulation of T2 512
Number of Trotter steps for QDrift simulation of tmix 1
Difference between Trotterized pulse_mix and reference pulse_mix: 4.680482060069453


/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1436620669.py:328: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1436620669.py:331: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1436620669.py:334: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_tem

In [103]:
cos_coh_fid = fid_coh_1 - fid_coh_3
sin_coh_fid = fid_coh_2 - fid_coh_4

savemat('ALA_truncR_FIDcos_QDrift512_cohT1T2_1Step.mat', {'FID_cos': cos_coh_fid})
savemat('ALA_truncR_FIDsin_QDrift512_cohT1T2_1Step.mat', {'FID_sin': sin_coh_fid})





In [107]:

#####Numerical experiment to check whether it is more accurate to implement the dissipative single channel at the beginning or at the end of mixing time:

def eff_QDriftChann_InitJump(List_gens,time_evol,NSteps):
    """
    Calculation of an effective QDrift channel. Based on the observation that the action of jump operators are in essence rare events 
    """
    Norm_ops, List_weights = Normalize_and_weightOps(List_gens)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    Eff_probs = np.zeros(len(List_gens))

    Eff_probs[0] = pks[0]**(NSteps) #probability of sampling coherent evolution only

    for i in range(1,len(Eff_probs)):
        Eff_probs[i] = pks[0]**(NSteps-1) * pks[i] #probability of sampling a circuit with NSteps-1 layers of H0 and one jump operator
    
    #Glob_Chann = Eff_probs[0]*expm(-1j*List_gens[0]*time_evol)
    Glob_Chann = Eff_probs[0]*expm(-1j*Norm_ops[0]*time_evol*Gamma)

    for i in range(1,len(Eff_probs)):
        Glob_Chann = Glob_Chann + NSteps*Eff_probs[i]*expm(-1j*Norm_ops[0]*(time_evol/NSteps)*Gamma*(NSteps-1))*expm(Norm_ops[i]*(time_evol/NSteps)*Gamma)

    #Glob_Chann =Glob_Chann + Eff_probs[0]

    #Glob_Chann = Glob_Chann+ProbSingleJumpLayers(List_gens,Eff_probs[1:],time_evol,NSteps)

    #Glob_Chann = Glob_Chann+ProbSingleJumpLayers(Norm_ops,Eff_probs[1:],time_evol*Gamma,NSteps)
    return Glob_Chann

def eff_QDriftChann_FinJump(List_gens,time_evol,NSteps):
    """
    Calculation of an effective QDrift channel. Based on the observation that the action of jump operators are in essence rare events 
    """
    Norm_ops, List_weights = Normalize_and_weightOps(List_gens)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    Eff_probs = np.zeros(len(List_gens))

    Eff_probs[0] = pks[0]**(NSteps) #probability of sampling coherent evolution only

    for i in range(1,len(Eff_probs)):
        Eff_probs[i] = pks[0]**(NSteps-1) * pks[i] #probability of sampling a circuit with NSteps-1 layers of H0 and one jump operator
    
    #Glob_Chann = Eff_probs[0]*expm(-1j*List_gens[0]*time_evol)
    Glob_Chann = Eff_probs[0]*expm(-1j*Norm_ops[0]*time_evol*Gamma)

    for i in range(1,len(Eff_probs)):
        Glob_Chann = Glob_Chann + NSteps*Eff_probs[i]*expm(Norm_ops[i]*(time_evol/NSteps)*Gamma)*expm(-1j*Norm_ops[0]*(time_evol/NSteps)*Gamma*(NSteps-1))

    #Glob_Chann =Glob_Chann + Eff_probs[0]

    #Glob_Chann = Glob_Chann+ProbSingleJumpLayers(List_gens,Eff_probs[1:],time_evol,NSteps)

    #Glob_Chann = Glob_Chann+ProbSingleJumpLayers(Norm_ops,Eff_probs[1:],time_evol*Gamma,NSteps)
    return Glob_Chann




In [111]:
rho_init = expm(-1j*Lx*np.pi/2)@rho0.flatten()

ref_rho_t = expm((-1j*H_ala+sum(JumpOps))*tmix)@rho_init

orig_chann = eff_QDriftChann_OneJump([H_ala]+JumpOps,tmix, 2631)

InitJ_chann = eff_QDriftChann_InitJump([H_ala]+JumpOps,tmix,2631)
FinJ_chann = eff_QDriftChann_FinJump([H_ala]+JumpOps,tmix,2631)

print("Fidelity of Init channel is: ", np.dot(np.conjugate(ref_rho_t),InitJ_chann@rho_init))
print("Fidelity of Fin channel is: ", np.dot(np.conjugate(ref_rho_t),FinJ_chann@rho_init))
print("Fidelity of the one-jump effective channel is: ", np.dot(np.conjugate(ref_rho_t),orig_chann@rho_init))





Fidelity of Init channel is:  (0.6854236443419044+3.833063756297431e-14j)
Fidelity of Fin channel is:  (0.6854236443419044+3.833063756297431e-14j)
Fidelity of the one-jump effective channel is:  (0.7060071383742876+5.317215139842124e-14j)


# Coherent T1/T2 time evolution + jump at beginning of evolution

In [123]:
Ntmix = 2631

fid_coh_1, fid_coh_2, fid_coh_3, fid_coh_4 =GenFID_TmixInitJumpQDrift_cohT1T2(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)

Number of Trotter steps for deterministic simulation of T1 512
Number of Trotter steps for deterministic simulation of T2 512
Number of Trotter steps for QDrift simulation of tmix 2631
Difference between Trotterized pulse_mix and reference pulse_mix: 3.4057490874963046


/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1202082691.py:453: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1202082691.py:456: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1202082691.py:459: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_tem

In [124]:
cos_coh_fid = fid_coh_1 - fid_coh_3
sin_coh_fid = fid_coh_2 - fid_coh_4

savemat('ALA_truncR_FIDcos_QDrift512_cohT1T2_initJump.mat', {'FID_cos': cos_coh_fid})
savemat('ALA_truncR_FIDsin_QDrift512_cohT1T2_initJump.mat', {'FID_sin': sin_coh_fid})


####There are only quantitative changes in a couple of signals with respect to the coherent T1/T2 evolution + qdrift for Tmix timescale...

#for resource efficiency purposes, the best compromise with accuracy corresponds to effective QDrift channel+coherent T1/T2 evolution

#Next step: how to Trotterize the coherent evolution

# Coherent T1/T2 time evolution + 2 step QDrift channel



In [118]:
Ntmix = 2


fid_coh_1, fid_coh_2, fid_coh_3, fid_coh_4 = GenFID_TmixQDrift_cohT1T2(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)


Number of Trotter steps for deterministic simulation of T1 512
Number of Trotter steps for deterministic simulation of T2 512
Number of Trotter steps for QDrift simulation of tmix 2
Difference between Trotterized pulse_mix and reference pulse_mix: 4.679913226275962


/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1436620669.py:328: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1436620669.py:331: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1436620669.py:334: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_tem

In [119]:
cos_coh_fid = fid_coh_1 - fid_coh_3
sin_coh_fid = fid_coh_2 - fid_coh_4


savemat('ALA_truncR_FIDcos_QDrift512_cohT1T2_2Step.mat', {'FID_cos': cos_coh_fid})
savemat('ALA_truncR_FIDsin_QDrift512_cohT1T2_2Step.mat', {'FID_sin': sin_coh_fid})


In [117]:
List_weights/sum(List_weights)

array([9.99955205e-01, 1.53581459e-05, 1.48248770e-05, 1.46115694e-05])

# Simplifying the coarse-graining of the QDrift channel

In [94]:
Ntmix = 1

fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4 = GenFID_TmixQDriftsignals(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)

Number of Trotter steps for deterministic simulation of T1 512
Number of Trotter steps for deterministic simulation of T2 512
Number of Trotter steps for QDrift simulation of tmix 1
Difference between Trotterized pulse_mix and reference pulse_mix: 4.680482060069453


/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/2501787718.py:235: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/2501787718.py:238: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/2501787718.py:241: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_tem

In [1]:

cos_fid = fid_temp_1 - fid_temp_3
sin_fid = fid_temp_2 - fid_temp_4


savemat('ALA_truncR_FIDcos_QDrift512_1step.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_QDrift512_1step.mat', {'FID_sin': sin_fid})

NameError: name 'fid_temp_1' is not defined

# Approximating the space of trajectories to a single jump operator for the whole protocol

In [14]:

def eff_QDriftChann_FinJump(List_gens,time_evol,NSteps):
    """
    Calculation of an effective QDrift channel. Based on the observation that the action of jump operators are in essence rare events 
    """
    Norm_ops, List_weights = Normalize_and_weightOps(List_gens)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    Eff_probs = np.zeros(len(List_gens))

    Eff_probs[0] = pks[0]**(NSteps) #probability of sampling coherent evolution only

    for i in range(1,len(Eff_probs)):
        Eff_probs[i] = pks[0]**(NSteps-1) * pks[i] #probability of sampling a circuit with NSteps-1 layers of H0 and one jump operator
    
    #Glob_Chann = Eff_probs[0]*expm(-1j*List_gens[0]*time_evol)
    Glob_Chann = Eff_probs[0]*expm(-1j*Norm_ops[0]*time_evol*Gamma)

    for i in range(1,len(Eff_probs)):
        Glob_Chann = Glob_Chann + NSteps*Eff_probs[i]*expm(Norm_ops[i]*(time_evol/NSteps)*Gamma)*expm(-1j*Norm_ops[0]*(time_evol/NSteps)*Gamma*(NSteps-1))

    #Glob_Chann =Glob_Chann + Eff_probs[0]

    #Glob_Chann = Glob_Chann+ProbSingleJumpLayers(List_gens,Eff_probs[1:],time_evol,NSteps)

    #Glob_Chann = Glob_Chann+ProbSingleJumpLayers(Norm_ops,Eff_probs[1:],time_evol*Gamma,NSteps)
    return Glob_Chann


def GenFID_JumpQDriftEnd_cohT1T2(Ham,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    
    R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    pulse_mix = expm((-1j*Ham)*tmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90y@pulse_mix@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_mix@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_mix@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_mix@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)

            Ldt2 = eff_QDriftChann_FinJump([Ham]+List_jumps,(j+1)*dt2,j+1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            Ldt2 = eff_QDriftChann_FinJump([Ham]+List_jumps,(j+1)*dt2,j+1)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            Ldt2 = eff_QDriftChann_FinJump([Ham]+List_jumps,(j+1)*dt2,j+1)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            Ldt2 = eff_QDriftChann_FinJump([Ham]+List_jumps,(j+1)*dt2,j+1)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4

In [15]:
T1 = Tpts1*dt1
T2 = Tpts2*dt2
Ntmix = 2631

In [21]:
###Loadiing remotely-processed data...

f = open('./data/ALA_FinJump_cohEvol.pk','rb')
dat = pickle.load(f)

fid_1 = dat['fid_1']
fid_2 = dat['fid_2']
fid_3 = dat['fid_3']
fid_4 = dat['fid_4']


In [23]:

cos_fid = fid_1 - fid_3
sin_fid = fid_2 - fid_4


savemat('ALA_truncR_FIDcos_FinJump_coh.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_FinJump_coh.mat', {'FID_sin': sin_fid})

In [17]:

#fid_1, fid_2, fid_3, fid_4 = GenFID_JumpQDriftEnd_cohT1T2(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)

# Purely dissipative free evolution during mixing time

In [ ]:
#####There is a qualitative difference between the reproduced spectrum simulating the dissipation channels only during the mixing time

In [97]:
fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4 = GenFID_DissTmix(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)

Number of Trotter steps for deterministic simulation of T1 512
Number of Trotter steps for deterministic simulation of T2 512
Number of Trotter steps for QDrift simulation of tmix 1
Difference between pulse_mix and reference pulse_mix: 17.02697525169766


/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1436620669.py:420: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1436620669.py:423: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_42693/1436620669.py:426: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_tem

In [98]:
cos_fid = fid_temp_1 - fid_temp_3
sin_fid = fid_temp_2 - fid_temp_4

savemat('ALA_truncR_FIDcos_DissTmix.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_DissTmix.mat', {'FID_sin': sin_fid})


# Fidelity of QDrift channel with respect to exact propagation





In [129]:
np.kron(np.array([0,1]),np.array([0,1]))

array([0, 0, 0, 1])

In [128]:
import scipy.linalg


scipy.linal

<module 'scipy.linalg' from '/Users/luismartinezmartinez/BQSKit/lib/python3.11/site-packages/scipy/linalg/__init__.py'>

In [130]:
np.kron(np.array([[0,1],[0,0]]),np.eye(2))

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [131]:
####Decomposing the density matrix of a maximally entangled state in an extended Hilbert space, using the Sz eigenbasis

#We can use our pipeline based on an orthonormalized Pauli basis to simplify the calculation:

def Get_RT(Channel):
    ###Dimensionality of the channel:
    dim_chan = Channel.shape[0]

    Phi_plus = np.zeros(dim_chan**2)

    for i in range(dim_chan):
        vec = np.zeros(dim_chan)
        vec[i] = 1.0
        Phi_plus+=np.kron(vec,vec)

    #Extended channel:
    Chann_I = np.kron(Channel,np.eye(dim_chan))

    return (1.0/dim_chan)*Chann_I@Phi_plus




In [132]:
Ref_Ext_den = Get_RT(expm((-1j*H_ala+Rtrunc)*0.1))

In [161]:
####The calculation of the "purified" density matrix in the Pauli basis was simple but the fidelity of density matrices is not thats simple...
####Construction of the density matrix...


def Get_DenMat_vromVect(Vect,NormBasis):
    """
    Get the density matrix in the {S_{z}} eigenbasis starting from its vector representation in the Pauli Spinach's basis
    """
    dim_orig = len(NormBasis)
    nqubs = int(np.log2(dim_orig)/2)

    DenMat = np.zeros([2**(2*nqubs),2**(2*nqubs)],dtype=complex)

    for i in range(dim_orig):
        vec= np.zeros(dim_orig)
        vec[i] = 1.0
        ext_vec = np.kron(vec,vec)
        weight = np.vdot(ext_vec,Vect)
        Bas_element_mat_form = of.get_sparse_operator(NormBasis[i],n_qubits=nqubs)

        DenMat+=weight*np.kron(Bas_element_mat_form.toarray(),Bas_element_mat_form.toarray())


    return DenMat




In [162]:
Ref_DenMat = Get_DenMat_vromVect(Ref_Ext_den,Normbasis_ala)

In [163]:
np.shape(Ref_DenMat)

(256, 256)